In [6]:
import networkx as nx
import random
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[15,7]

In [7]:
g = nx.read_edgelist("complete_networld_cap.edgelist", create_using = nx.DiGraph())

In [8]:
def random_walk(g):
    edge_count = {}
    for i in range(5000):
        start = str(np.random.randint(1,12982))
        status = True
        while status == True:
            for j in range(1,9):
                node_neighbors = list(g.neighbors(start))
                if len(node_neighbors) != 0:
                    N = random.choice(list(g.neighbors(start)))
                    pair = start + ' ' + N

                    try: 
                        edge_count[pair] += 1
                    except KeyError as e:
                        edge_count[pair] = 1   
                    start = N

                else:
                    status = False
            status = False 
    return edge_count
            

In [9]:
def avg_list(alist):
    avg_list = [sum(elem)/len(elem) for elem in zip(*alist)]
    
    return avg_list

In [41]:
def res_test(edgelist):
    
    #avg_weak_comp_list, avg_strong_comp_list, avg_total_comp_list, avg_edge_percentage_value_list = [], [], [], []
    
    #for run in range(runs):
        #print("Run: ", run+1)
    
    g = nx.read_edgelist(edgelist, create_using = nx.DiGraph())
    g_edge_cap = nx.get_edge_attributes(g,'capacity')
    g_edge_cars = nx.get_edge_attributes(g,'cars')
    edge_count = random_walk(g)
    ordered_edges = sorted(edge_count, key=edge_count.get, reverse=True)
    comp_count, weakly_comp_count, strongly_comp_count, edge_percentage = [], [], [], []
    weakly_comp_count.append(nx.number_weakly_connected_components(g))
    strongly_comp_count.append(nx.number_strongly_connected_components(g))
    comp_count.append(weakly_comp_count[0] + strongly_comp_count[0])
    edge_percentage.append(0/len(g.edges())*100)
    total_edge_count = len(g.edges())
    
    for index, elem in enumerate(ordered_edges):
        elem = elem.split(' ')
        start_node = elem[0]
        end_node = elem[1]
        cars = g_edge_cars[start_node,end_node]
        g.remove_edge(start_node,end_node)
        start_node_neighbors = g.successors(start_node)
        neighbors_w_path = {}
        
        for successor in start_node_neighbors:
            have_path = nx.has_path(g, successor, end_node)
            if have_path:
                neighbors_w_path[successor] = start_node, successor
                #print(neighbors_w_path[successor])
                #print(neighbors_w_path[successor][1])
            else:
                continue
        
        if len(neighbors_w_path) != 0:
            cars_frac = cars/len(neighbors_w_path)
            fail_set = set()
            redistribute_cars(start_node, cars_frac, neighbors_w_path,g_edge_cars, g_edge_cap, fail_set)
            
        else:
            print("Fail Option 1")
            
        '''
        if i % 100 == 0:

            weakly_comp = nx.number_weakly_connected_components(g)
            strongly_comp = nx.number_strongly_connected_components(g)

            comp_count.append(weakly_comp + strongly_comp)
            weakly_comp_count.append(weakly_comp)
            strongly_comp_count.append(strongly_comp)

            edge_percentage_value = i/total_edge_count * 100
            edge_percentage.append(edge_percentage_value)

        avg_weak_comp_list.append(weakly_comp_count)
        avg_strong_comp_list.append(strongly_comp_count)
        avg_total_comp_list.append(comp_count)
        avg_edge_percentage_value_list.append(edge_percentage)

        final_weak = avg_list(avg_weak_comp_list)
        final_strong =  avg_list(avg_strong_comp_list)
        final_total = avg_list(avg_total_comp_list)
        final_percentage = avg_list(avg_edge_percentage_value_list)

        plt.plot(final_percentage, final_weak, label="weak")
        plt.plot(final_percentage, final_strong, label="strong")
        plt.plot(final_percentage, final_total, label="total")
        plt.xlabel('Edge Removal Percentage')
        plt.ylabel('Number of Components')
        plt.legend()
'''

In [57]:
def redistribute_cars(start_node, cars_frac, neighbor_w_path, g_edge_cars, g_edge_cap, fail_set):
    for successor in neighbor_w_path:
        end_node = neighbor_w_path[successor][1]
        if g_edge_cars[start_node, end_node] + cars_frac > g_edge_cap[start_node,end_node]:
            if successor in fail_set:
                continue
            else:
                fail_set.add(successor)

                if len(fail_set) != len(neighbor_w_path):
                    extra_cars = cars_frac/(len(neighbor_w_path)-len(fail_set))
                    print(start_node, extra_cars, neighbor_w_path, end_node, fail_set)
                    print(g_edge_cap[start_node, end_node])
                    print(g_edge_cars[start_node, end_node])
                    redistribute_cars(start_node, extra_cars, neighbor_w_path, g_edge_cars, g_edge_cap, fail_set)
                else:
                    print("Fail Option 2")
        else:
            g_edge_cars[start_node,end_node] += cars_frac

In [58]:
res_test("complete_networld_cap.edgelist")

Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail O

4351 600.0 {'3767': ('4351', '3767'), '4347': ('4351', '4347')} 3767 {'3767'}
900.0
450.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
2500 327.0 {'2501': ('2500', '2501'), '865': ('2500', '865')} 2501 {'2501'}
600.0
300.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
12778 700.0 {'11076': ('12778', '11076'), '12775': ('12778', '12775')} 11076 {'11076'}
900.0
450.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
6245 327.0 {'6169': ('6245', '6169'), '7008': ('6245', '7008')} 6169 {'6169'}
600.0
300.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
7586 600.0 {'7592': ('7586', '7592'), '7584': ('7586', '7584')} 7584 {'7584'}
900.0
450.0
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 2
Fail Option

300.0
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
5334 800.0 {'984': ('5334', '984'), '652': ('5334', '652')} 652 {'652'}
1309
1054.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
12352 600.0 {'873': ('12352', '873'), '12363': ('12352', '12363')} 873 {'873'}
900.0
450.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
927 600.0 {'2843': ('927', '2843'), '928': ('927', '928')} 928 {'928'}
900.0
450.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
431 163.5 {'10036': ('431', '10036'), '441': ('431', '441'), '10020': ('431', '10020')} 441 {'441'}
900.0
777.0
431 163.5 {'10036': ('431', '10036'), '441': ('431', '441'), '10020': ('431', 

Fail Option 1
11061 800.0 {'38': ('11061', '38'), '41': ('11061', '41')} 38 {'38'}
1309
1054.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
5899 700.0 {'6031': ('5899', '6031'), '758': ('5899', '758')} 758 {'758'}
1309
654
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Optio

654
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
12385 334.0 {'12367': ('12385', '12367'), '11035': ('12385', '11035')} 11035 {'11035'}
900.0
668.0
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
6046 291.6666666666667 {'748': ('6046', '748'), '6039': ('6046', '6039'), '3653': ('6046', '3653')} 748 {'748'}
1309
1204.0
6046 291.6666666666667 {'748': ('6046', '748'), '6039': ('6046', '6039'), '3653': ('6046', '3653')} 6039 {'748', '6039'}
300.0
250.0
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
11474 700.0 {'11475': ('11474', '11475'), '489': ('11474', '489')} 489 {'489'}
1309
654
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
1289 233.33333333333334 {'1663': ('1289', '1663'), '1287': ('1289', '1287'), '2567': ('1289', '2567')} 1287 {'1287'}
900.0
450.0
1289 466.6666666666667 {'1663':

...

300.0
150.0
500 250.0 {'12128': ('500', '12128'), '498': ('500', '498')} 12128 {'12128'}
1309
1281.0
Fail Option 1
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
86 6.660384469652631 {'101': ('86', '101'), '233': ('86', '233'), '191': ('86', '191'), '7': ('86', '7'), '11987': ('86', '11987'), '12003': ('86', '12003'), '87': ('86', '87'), '11990': ('86', '11990'), '120': ('86', '120'), '310': ('86', '310')} 233 {'233'}
600.0
599.4346022687369
86 0.8325480587065789 {'101': ('86', '101'), '233': ('86', '233'), '191': ('86', '191'), '7': ('86', '7'), '11987': ('86', '11987'), '12003': ('86', '12003'), '87': ('86', '87'), '11990': ('86', '11990'), '120': ('86', '120'), '310': ('86', '310')} 7 {'233', '7'}
1309
1308.838305537807
86 0.1189354369580827 {'101': ('86', '101'), '233': ('86', '233'), '191': ('86', '191'), '7': ('86', '7'), '11987': ('86', '11987'), '12003': ('86', '12003'), '87': ('86', '87'), '11990': ('86', '11990'), '120': ('86', '120'), '31

Fail Option 1
Fail Option 1
546 51.55 {'6927': ('546', '6927'), '548': ('546', '548'), '6916': ('546', '6916'), '592': ('546', '592'), '29': ('546', '29')} 548 {'548'}
900.0
827.0
546 68.73333333333333 {'6927': ('546', '6927'), '548': ('546', '548'), '6916': ('546', '6916'), '592': ('546', '592'), '29': ('546', '29')} 592 {'592', '548'}
900.0
878.55
546 34.36666666666667 {'6927': ('546', '6927'), '548': ('546', '548'), '6916': ('546', '6916'), '592': ('546', '592'), '29': ('546', '29')} 6927 {'6927', '592', '548'}
1309
1288.75
546 34.36666666666667 {'6927': ('546', '6927'), '548': ('546', '548'), '6916': ('546', '6916'), '592': ('546', '592'), '29': ('546', '29')} 6916 {'6927', '6916', '592', '548'}
1309
1288.75
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
183 20.756381337808495 {'11437': ('183', '11437'), '10428': ('183', '10428'), '306': ('183', '306'), '323': ('183', '323'), '161': ('183', '161'), '292': ('183', '292'), '154': ('183', '154')} 11437 {'11437'}
1309
1291.971

1309
1237.6961038961038
285 102.51659451659452 {'58': ('285', '58'), '22': ('285', '22'), '338': ('285', '338'), '7927': ('285', '7927'), '7924': ('285', '7924'), '7917': ('285', '7917')} 7917 {'7927', '7924', '58', '7917', '22'}
1309
1288.954401154401
Fail Option 1
Fail Option 2
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
210 15.233449470044214 {'214': ('210', '214'), '28': ('210', '28'), '11173': ('210', '11173'), '215': ('210', '215'), '190': ('210', '190'), '223': ('210', '223'), '325': ('210', '325'), '206': ('210', '206'), '309': ('210', '309')} 214 {'214'}
600.0
598.17129985755
210 2.1762070671491736 {'214': ('210', '214'), '28': ('210', '28'), '11173': ('210', '11173'), '215': ('210', '215'), '190': ('210', '190'), '223': ('210', '223'), '325': ('210', '325'), '206': ('210', '206'), '309': ('210', '309')} 215 {'214', '215'}
900.0
892.8083618431835
210 0.3627011778581956

5 0.22705460430612337 {'78': ('5', '78'), '273': ('5', '273'), '77': ('5', '77'), '11698': ('5', '11698'), '254': ('5', '254'), '264': ('5', '264'), '752': ('5', '752'), '893': ('5', '893'), '255': ('5', '255'), '11696': ('5', '11696')} 255 {'273', '77', '11698', '264', '255', '78', '752', '254', '893'}
1309
1308.9114083551224
Fail Option 2
415 1260.3333333333333 {'9106': ('415', '9106'), '9107': ('415', '9107')} 9106 {'9106'}
1309
1144.5
Fail Option 2
Fail Option 1
527 182.6 {'528': ('527', '528'), '5804': ('527', '5804'), '72': ('527', '72')} 528 {'528'}
900.0
891.6
527 365.2 {'528': ('527', '528'), '5804': ('527', '5804'), '72': ('527', '72')} 5804 {'528', '5804'}
1309
1278.1999999999998
Fail Option 2
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
Fail Option 1
798 1027.0 {'6690': ('798', '6690'), '6703': ('798', '6703')} 6690 {'6690'}
1309
1308.0
Fa

Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
112 89.75257936507937 {'12': ('112', '12'), '10966': ('112', '10966'), '11678': ('112', '11678'), '199': ('112', '199'), '11675': ('112', '11675')} 12 {'12'}
1309
1249.0515873015872
112 29.917526455026458 {'12': ('112', '12'), '10966': ('112', '10966'), '11678': ('112', '11678'), '199': ('112', '199'), '11675': ('112', '11675')} 10966 {'10966', '12'}
1309
1249.0515873015872
112 44.876289682539685 {'12': ('112', '12'), '10966': ('112', '10966'), '11678': ('112', '11678'), '199': ('112', '199'), '11675': ('112', '11675')} 11678 {'11678', '10966', '12'}
1309
1278.9691137566138
112 44.876289682539685 {'12': ('112', '12'), '10966': ('112', '10966'), '11678': ('112', '11678'), '199': ('112', '199'), '11675': ('112', '11675')} 11675 {'11678', '10966', '12', '11675'}
1309
1278.9691137566138
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 1
Fail Option 2
11285 66.66666666666667 {'526': ('11285', '526'), '11274': ('11285', '11274'), '